In [ ]:
# =============================================================================
# ANÁLISIS DE SENTIMIENTOS PARA COMENTARIOS DE INSTAGRAM
# Modelo de Machine Learning Simple para Google Colab
# =============================================================================

# %% [markdown]
# # 📦 Paso 1: Instalar e importar librerías necesarias

# %%
# Instalar librerías (si es necesario en Colab)
# !pip install pandas scikit-learn numpy matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configurar visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

In [ ]:
# %% [markdown]
# # 📂 Paso 2: Cargar los datos

# %%
# Para Google Colab - Subir archivo
from google.colab import files
uploaded = files.upload()

# Cargar el CSV
df = pd.read_csv('comentarios_instagram.csv')

In [ ]:
# Mostrar información básica
print("📊 Información del Dataset:")
print(f"Número de filas: {len(df)}")
print(f"Número de columnas: {len(df.columns)}")
print("\nPrimeras 5 filas:")
df.head()

In [ ]:
# %% [markdown]
# # 🔍 Paso 3: Exploración de datos

# %%
# Ver estructura de datos
print("📋 Información del DataFrame:")
print(df.info())
print("\n📈 Estadísticas descriptivas:")
print(df.describe())

In [ ]:
# Verificar valores nulos
print("\n❓ Valores nulos por columna:")
print(df.isnull().sum())

In [ ]:
# Ver algunos comentarios de ejemplo
print("\n💬 Ejemplos de comentarios:")
for i in range(5):
    print(f"{i+1}. {df['texto_comentario'].iloc[i][:100]}...")

In [ ]:
# %% [markdown]
# # 🏷️ Paso 4: Crear etiquetas de sentimiento

In [ ]:
# %%
def asignar_sentimiento(row):
    """
    Función simple para asignar sentimiento basado en palabras positivas/negativas
    """
    pos = row['palabras_positivas']
    neg = row['palabras_negativas']
    ratio = row['ratio_pos_neg']
    
    # Reglas simples de clasificación
    if pos > neg:
        return 'positivo'
    elif neg > pos:
        return 'negativo'
    elif ratio > 1:
        return 'positivo'
    elif ratio > 0 and ratio < 1:
        return 'negativo'
    else:
        return 'neutral'

In [ ]:
# Crear columna de sentimiento
df['sentimiento'] = df.apply(asignar_sentimiento, axis=1)

In [ ]:
# Ver distribución de sentimientos
print("📊 Distribución de sentimientos:")
print(df['sentimiento'].value_counts())
print("\n📈 Porcentajes:")
print(df['sentimiento'].value_counts(normalize=True) * 100)

In [ ]:
# Visualizar distribución
plt.figure(figsize=(8, 5))
df['sentimiento'].value_counts().plot(kind='bar', color=['#2ecc71', '#e74c3c', '#95a5a6'])
plt.title('Distribución de Sentimientos en Comentarios')
plt.xlabel('Sentimiento')
plt.ylabel('Cantidad')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# %% [markdown]
# # 🧹 Paso 5: Preprocesamiento de texto

# %%
def limpiar_texto(texto):
    """
    Función para limpiar y normalizar el texto
    """
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Eliminar URLs
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto)
    
    # Eliminar menciones (@usuario)
    texto = re.sub(r'@\w+', '', texto)
    
    # Eliminar hashtags
    texto = re.sub(r'#\w+', '', texto)
    
    # Eliminar números
    texto = re.sub(r'\d+', '', texto)
    
    # Eliminar puntuación excesiva
    texto = re.sub(r'[^\w\s]', ' ', texto)
    
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

# Aplicar limpieza
df['texto_limpio'] = df['texto_comentario'].apply(limpiar_texto)

In [ ]:
print("✅ Texto limpiado")
print("\n🔍 Ejemplos de texto antes y después:")
for i in range(3):
    print(f"\nOriginal: {df['texto_comentario'].iloc[i]}")
    print(f"Limpio: {df['texto_limpio'].iloc[i]}")

In [ ]:
# %% [markdown]
# # 📊 Paso 6: Crear características (Feature Engineering)

# %%
# Crear características adicionales
df['longitud_comentario'] = df['texto_comentario'].str.len()
df['num_exclamaciones'] = df['texto_comentario'].str.count('!')
df['num_interrogaciones'] = df['texto_comentario'].str.count('\?')
df['tiene_emoji'] = df['num_emojis'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# Crear diccionario de palabras positivas y negativas
palabras_positivas = [
    'excelente', 'bueno', 'buena', 'genial', 'increíble', 'delicioso',
    'rico', 'perfecto', 'recomiendo', 'mejor', 'encanta', 'bonito',
    'agradable', 'rápido', 'limpio', 'fresco', 'calidad', 'super'
]

palabras_negativas = [
    'malo', 'mala', 'pésimo', 'terrible', 'horrible', 'lento', 'caro',
    'sucio', 'frío', 'crudo', 'nunca', 'peor', 'decepción', 'mal',
    'mediocre', 'regular', 'tardó', 'esperé'
]

In [ ]:
def contar_palabras_sentimiento(texto, palabras):
    """Contar palabras de sentimiento en el texto"""
    texto_lower = texto.lower()
    return sum(1 for palabra in palabras if palabra in texto_lower)

# Aplicar conteo
df['count_pos'] = df['texto_limpio'].apply(lambda x: contar_palabras_sentimiento(x, palabras_positivas))
df['count_neg'] = df['texto_limpio'].apply(lambda x: contar_palabras_sentimiento(x, palabras_negativas))

print("✅ Características creadas")
print("\nNuevas características:")
print(df[['texto_comentario', 'count_pos', 'count_neg', 'num_exclamaciones']].head())

In [ ]:
# %% [markdown]
# # 🤖 Paso 7: Preparar datos para el modelo

# %%
# Separar características y etiquetas
X = df['texto_limpio']
y = df['sentimiento']

In [ ]:
# Codificar etiquetas
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Dividir en entrenamiento y prueba (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"📊 Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"📊 Tamaño del conjunto de prueba: {len(X_test)}")

In [ ]:
# Vectorizar el texto usando TF-IDF
vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"\n✅ Texto vectorizado")
print(f"Forma de X_train: {X_train_vec.shape}")
print(f"Forma de X_test: {X_test_vec.shape}")

In [ ]:
# %% [markdown]
# # 🎯 Paso 8: Entrenar modelos simples

# %%
# Modelo 1: Naive Bayes
print("=" * 50)
print("🤖 MODELO 1: NAIVE BAYES")
print("=" * 50)

nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)

# Predicciones
y_pred_nb = nb_model.predict(X_test_vec)

# Evaluación
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"\n✅ Precisión Naive Bayes: {accuracy_nb:.2%}")

print("\n📊 Reporte de clasificación:")
print(classification_report(y_test, y_pred_nb, 
                          target_names=label_encoder.classes_))

In [ ]:
# %%
# Modelo 2: Regresión Logística
print("=" * 50)
print("🤖 MODELO 2: REGRESIÓN LOGÍSTICA")
print("=" * 50)

lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_vec, y_train)

# Predicciones
y_pred_lr = lr_model.predict(X_test_vec)

# Evaluación
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"\n✅ Precisión Regresión Logística: {accuracy_lr:.2%}")

print("\n📊 Reporte de clasificación:")
print(classification_report(y_test, y_pred_lr, 
                          target_names=label_encoder.classes_))

In [ ]:
# %% [markdown]
# # 📈 Paso 9: Visualizar resultados

# %%
# Matriz de confusión para el mejor modelo
from sklearn.metrics import ConfusionMatrixDisplay

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

In [ ]:
# Naive Bayes
cm_nb = confusion_matrix(y_test, y_pred_nb)
disp_nb = ConfusionMatrixDisplay(confusion_matrix=cm_nb, 
                                  display_labels=label_encoder.classes_)
disp_nb.plot(ax=axes[0], cmap='Blues')
axes[0].set_title(f'Naive Bayes (Precisión: {accuracy_nb:.2%})')

In [ ]:
# Regresión Logística
cm_lr = confusion_matrix(y_test, y_pred_lr)
disp_lr = ConfusionMatrixDisplay(confusion_matrix=cm_lr, 
                                  display_labels=label_encoder.classes_)
disp_lr.plot(ax=axes[1], cmap='Greens')
axes[1].set_title(f'Regresión Logística (Precisión: {accuracy_lr:.2%})')

plt.tight_layout()
plt.show()

In [ ]:
# %%
# Comparación de modelos
modelos = ['Naive Bayes', 'Regresión Logística']
precisiones = [accuracy_nb, accuracy_lr]

plt.figure(figsize=(8, 5))
bars = plt.bar(modelos, precisiones, color=['#3498db', '#2ecc71'])
plt.ylim(0, 1)
plt.ylabel('Precisión')
plt.title('Comparación de Modelos')

# Agregar valores en las barras
for bar, precision in zip(bars, precisiones):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{precision:.2%}', ha='center')

plt.show()

In [ ]:
# %% [markdown]
# # 🔮 Paso 10: Hacer predicciones con nuevos comentarios

# %%
def predecir_sentimiento(texto, modelo=lr_model, vectorizer=vectorizer):
    """
    Función para predecir el sentimiento de un nuevo comentario
    """
    # Limpiar texto
    texto_limpio = limpiar_texto(texto)
    
    # Vectorizar
    texto_vec = vectorizer.transform([texto_limpio])
    
    # Predecir
    prediccion = modelo.predict(texto_vec)[0]
    probabilidades = modelo.predict_proba(texto_vec)[0]
    
    # Decodificar etiqueta
    sentimiento = label_encoder.inverse_transform([prediccion])[0]
    
    # Obtener confianza
    confianza = max(probabilidades) * 100
    
    return sentimiento, confianza, probabilidades

In [ ]:
# Ejemplos de predicción
comentarios_nuevos = [
    "La comida estaba deliciosa y el servicio excelente!",
    "Terrible experiencia, no lo recomiendo para nada",
    "El lugar está bien, nada especial",
    "Me encantó! Definitivamente volveré 😍",
    "Pésimo servicio, esperé una hora y la comida llegó fría"
]

print("🔮 PREDICCIONES DE NUEVOS COMENTARIOS")
print("=" * 60)

In [ ]:
for comentario in comentarios_nuevos:
    sentimiento, confianza, probs = predecir_sentimiento(comentario)
    print(f"\n💬 Comentario: '{comentario}'")
    print(f"🎯 Sentimiento: {sentimiento.upper()}")
    print(f"📊 Confianza: {confianza:.1f}%")
    print(f"📈 Probabilidades: Neg={probs[0]:.2f}, Neu={probs[1]:.2f}, Pos={probs[2]:.2f}")
    print("-" * 60)

# %% [markdown]
# # 💾 Paso 11: Guardar el modelo

# %%
import pickle

In [ ]:
# Guardar el modelo y el vectorizador
with open('modelo_sentimientos.pkl', 'wb') as f:
    pickle.dump(lr_model, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("✅ Modelo guardado exitosamente")

In [ ]:
# Para cargar el modelo después:
# with open('modelo_sentimientos.pkl', 'rb') as f:
#     modelo_cargado = pickle.load(f)

# %% [markdown]
# # 🎨 Paso 12: Crear función completa para análisis

# %%
class AnalizadorSentimientos:
    """
    Clase completa para análisis de sentimientos
    """
    def __init__(self, modelo, vectorizer, label_encoder):
        self.modelo = modelo
        self.vectorizer = vectorizer
        self.label_encoder = label_encoder
        
    def analizar_comentario(self, comentario):
        """Analiza un comentario y retorna resultados detallados"""
        
        # Limpiar texto
        texto_limpio = limpiar_texto(comentario)
        
        # Vectorizar
        texto_vec = self.vectorizer.transform([texto_limpio])
        
        # Predecir
        prediccion = self.modelo.predict(texto_vec)[0]
        probabilidades = self.modelo.predict_proba(texto_vec)[0]
        
        # Decodificar
        sentimiento = self.label_encoder.inverse_transform([prediccion])[0]
        
        # Crear diccionario de resultados
        resultados = {
            'comentario_original': comentario,
            'comentario_limpio': texto_limpio,
            'sentimiento': sentimiento,
            'confianza': max(probabilidades) * 100,
            'probabilidades': {
                'negativo': probabilidades[0],
                'neutral': probabilidades[1],
                'positivo': probabilidades[2]
            }
        }
        
        return resultados
    
    def analizar_multiples(self, lista_comentarios):
        """Analiza múltiples comentarios"""
        resultados = []
        for comentario in lista_comentarios:
            resultados.append(self.analizar_comentario(comentario))
        return pd.DataFrame(resultados)

# Crear instancia del analizador
analizador = AnalizadorSentimientos(lr_model, vectorizer, label_encoder)

# Probar el analizador
comentario_prueba = "Este restaurante es increíble, la comida está deliciosa!"
resultado = analizador.analizar_comentario(comentario_prueba)

print("🎯 ANÁLISIS COMPLETO")
print("=" * 50)
for key, value in resultado.items():
    print(f"{key}: {value}")

In [ ]:
# %% [markdown]
# # 📊 Resumen y Conclusiones

# %%
print("=" * 60)
print("📊 RESUMEN DEL MODELO DE ANÁLISIS DE SENTIMIENTOS")
print("=" * 60)

print(f"""
📈 Métricas del Modelo:
   - Precisión Naive Bayes: {accuracy_nb:.2%}
   - Precisión Regresión Logística: {accuracy_lr:.2%}
   - Mejor modelo: {'Regresión Logística' if accuracy_lr > accuracy_nb else 'Naive Bayes'}

📊 Distribución del Dataset:
   - Total de comentarios: {len(df)}
   - Positivos: {(df['sentimiento'] == 'positivo').sum()} ({(df['sentimiento'] == 'positivo').sum()/len(df)*100:.1f}%)
   - Negativos: {(df['sentimiento'] == 'negativo').sum()} ({(df['sentimiento'] == 'negativo').sum()/len(df)*100:.1f}%)
   - Neutrales: {(df['sentimiento'] == 'neutral').sum()} ({(df['sentimiento'] == 'neutral').sum()/len(df)*100:.1f}%)

✅ Próximos pasos para mejorar:
   1. Balancear el dataset (hay muchos más neutrales)
   2. Agregar más características (bigramas, trigramas)
   3. Probar otros modelos (SVM, Random Forest, Redes Neuronales)
   4. Usar embeddings pre-entrenados (Word2Vec, BERT)
   5. Aumentar el tamaño del dataset
""")

print("\n🎉 ¡Modelo completado y listo para usar!")